In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Define a custom color palette using the provided hex codes
nelo_color_palette = ["#3679EA", "#E781CB", "#E7FE74", "#75BEF9", "#BFB5ED", "#5BC682", "#70DB73", "#EBCC81", "#2A4062"]

# Set the custom palette as the default
sns.set_palette(nelo_color_palette)

In [3]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", palette=nelo_color_palette, rc=custom_params)

In [12]:
# =========================
# 0) Setup & Configuration
# =========================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
import json
from pathlib import Path

# Suppress warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 140)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Configuration parameters
CONFIG = {
    # File paths
    "FILEPATH": "/Users/Jaime/hello-world/risk_analytics_case_2025.csv",
    
    # Column identifiers
    "TARGET_COL": "is_fpd",
    "UW_SCORE_COL": "acquisition_uw_score",
    
    # Columns to drop (post-application features)
    "DROP_POST_APP_COLS": [
        # Add any columns that occur after application
        # e.g., utilization metrics, limit changes, payment events
    ],
    
    # Missingness thresholds
    "LOW_MISS": 0.01,  # Drop if missing rate below this
    "HIGH_MISS": 0.99, # Drop if missing rate above this
    
    # Cross-validation parameters
    "N_SPLITS": 5,     # For target encoding
    
    # Modeling parameters
    "TRAIN_SIZE": 0.7,
    "MIN_SAMPLES_LEAF": 0.01,  # As fraction of total samples
    "MAX_DEPTH": 4,
    
    # Rule evaluation parameters
    "MIN_FPD_LIFT": 0.02,      # Minimum FPD rate increase vs baseline
    "MIN_LEAF_SIZE": 0.01,     # Minimum leaf size as fraction of data
    "MIN_FPD_REDUCTION": 0.002  # Minimum FPD reduction for adding rule
}

# Create output directories if they don't exist
for dir_name in ['plots', 'outputs']:
    Path(dir_name).mkdir(exist_ok=True)

print("Configuration loaded successfully.")

Configuration loaded successfully.


# 1. Data Loading and Initial Sanity Checks

Let's load the data and perform initial sanity checks on target and underwriting score columns.

In [13]:
# Load the dataset
df = pd.read_csv(CONFIG["FILEPATH"], index_col=0)

# Filter rows with non-missing target and UW score
valid_mask = df[CONFIG["TARGET_COL"]].notna() & df[CONFIG["UW_SCORE_COL"]].notna()
df = df[valid_mask].copy()

# Print basic statistics
print(f"Dataset shape: {df.shape}")
print(f"\nTarget (FPD) rate: {df[CONFIG['TARGET_COL']].mean():.2%}")
print("\nFirst few rows of the dataset:")
display(df.head())

Dataset shape: (182462, 217)

Target (FPD) rate: 25.00%

First few rows of the dataset:


,first_due_date_vintage,first_loan_principal,credit_limit,limit_utilization,uw_bucket,acquisition_uw_score,fraud_score,is_fpd,first_loan_type,acquisition_channel,bureau_1_credit_lines_in_use,bureau_1_credit_lines_total,bureau_1_current_credit_balance,bureau_1_credit_lines_past_due,bureau_1_current_balance_past_due,bureau_1_credit_lines_past12_months,bureau_1_credit_amount_past12_months,bureau_1_credit_lines_past24_months,bureau_1_credit_amount_past24_months,bureau_1_recent_credit_inquiries,bureau_1_max_external_credit_limit,bureau_1_active_lines,bureau_1_loan_types_count,bureau_1_avg_days_credit,bureau_1_oldest_account_duration,bureau_1_newest_account_duration,bureau_1_pl_credits_count,bureau_1_cc_credits_count,bureau_1_pn_credits_count,bureau_1_cl_credits_count,bureau_1_af_credits_count,bureau_1_cac_credits_count,bureau_1_max_external_active_limit,bureau_1_min_external_active_limit,bureau_1_total_external_active_limit,bureau_1_avg_loan_types,bureau_1_perc_active_lines,bureau_1_perc_lines_in_use,bureau_1_debt_ratio_max_limit,bureau_1_debt_ratio_total_limit,bureau_1_over_utilized,bureau_1_utilization,bureau_2_credit_lines_in_use,bureau_2_credit_lines_total,bureau_2_current_credit_balance,bureau_2_credit_lines_past_due,bureau_2_current_balance_past_due,bureau_2_credit_lines_past12_months,bureau_2_credit_amount_past12_months,bureau_2_credit_lines_past24_months,bureau_2_credit_amount_past24_months,bureau_2_recent_credit_inquiries,bureau_2_max_external_credit_limit,bureau_2_active_lines,bureau_2_loan_types_count,bureau_2_avg_days_credit,bureau_2_oldest_account_duration,bureau_2_newest_account_duration,bureau_2_pl_credits_count,bureau_2_cc_credits_count,...,history_pp_max_good_standing_strike,credits_pp_saldo_vencido,credits_total_limite_credito,credit_check_r_total_approved_amount,credit_check_soc_fin_ob_mult_months_since_earliest_credit_check,credits_tc_max_peor_atraso,credit_check_f_in_last_3_months_approved_amount,credit_check_f_in_last_12_months_approved_amount,credit_check_r_months_since_last_credit_check,credits_cc_max_peor_atraso,credits_tc_limite_credito,credit_check_microfinanciera_total_approved_amount,credit_check_sic_in_last_12_months,credit_check_compania_prestamo_personal_total_approved_amount,credits_total_active_monto_pagar,credit_check_r_in_last_3_months_approved_amount,history_tc_max_reported_events,credits_pp_credito_maximo,credit_check_f_total_approved_amount,credits_saldo_vencido,credits_tc_credito_maximo,credits_tc_saldo_vencido,history_tc_active_max_reported_events,credits_months_since_most_recent_fecha_peor_atraso,credits_pp_max_peor_atraso,credits_total_monto_pagar,history_pp_total_good_standing_reports,credit_check_microfinanciera_months_since_last_credit_check,credit_check_bancos_months_since_earliest_credit_check,credit_check_q_in_last_3_months_approved_amount,history_cc_active_max_reported_events,credit_check_q_months_since_earliest_credit_check,credit_check_bancos_months_since_last_credit_check,credit_check_soc_fin_de_obj_multiple_total_approved_amount,credit_check_sic_months_since_earliest_credit_check,credits_total_credito_maximo,credits_tc_active_credito_maximo,credits_pp_active_credito_maximo,vendor_1_telegram_privacy_status,vendor_1_whatsapp_privacy_status,vendor_1_registered_profiles,vendor_1_registered_ecommerce_profiles,vendor_1_registered_email_provider_profiles,vendor_1_registered_messaging_profiles,vendor_1_registered_professional_profiles,vendor_1_registered_social_media_profiles,vendor_1_number_of_photos_returned,vendor_1_google_registered,vendor_1_telegram_registered,vendor_1_whatsapp_registered,vendor_1_microsoft_registered,vendor_1_facebook_registered,vendor_1_instagram_registered,vendor_1_twitter_registered,vendor_1_amazon_registered,vendor_1_linkedin_registered,user_state,removed_nelo_app_between_first_loan_and_first_due_date,days_since_most_recent_app_event_prior_to_first_due_date,total_mobile_app_events_between_first_loan_and_first_due_date
user_uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

# 2. Column Governance and Missing Value Handling

We'll now:
1. Drop post-application columns
2. Remove columns with extreme missingness
3. Create missing value flags for remaining columns

In [14]:
# Drop post-application columns
if CONFIG["DROP_POST_APP_COLS"]:
    df = df.drop(columns=CONFIG["DROP_POST_APP_COLS"])
    print(f"Dropped {len(CONFIG['DROP_POST_APP_COLS'])} post-application columns")

# Calculate missingness rates
missingness = df.isnull().mean()

# Drop columns with extreme missingness
low_miss = missingness[missingness < CONFIG["LOW_MISS"]].index
high_miss = missingness[missingness > CONFIG["HIGH_MISS"]].index
df = df.drop(columns=list(low_miss) + list(high_miss))

print(f"\nDropped {len(low_miss)} columns with missingness < {CONFIG['LOW_MISS']}")
print(f"Dropped {len(high_miss)} columns with missingness > {CONFIG['HIGH_MISS']}")

# Create missing value flags
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
string_cols = df.select_dtypes(include=['object', 'category']).columns

# Exclude binary columns and special columns from flag creation
binary_cols = [col for col in numeric_cols 
               if df[col].nunique() == 2 and not df[col].isnull().any()]
special_cols = [CONFIG["TARGET_COL"], CONFIG["UW_SCORE_COL"]]
flag_eligible = [col for col in list(numeric_cols) + list(string_cols) 
                if col not in binary_cols + special_cols]

# Create missing flags
missing_flags = pd.DataFrame({
    f"{col}_missing": df[col].isnull().astype(int)
    for col in flag_eligible
})

# Add flags to dataframe
df = pd.concat([df, missing_flags], axis=1)

print(f"\nCreated {len(missing_flags.columns)} missing value flags")
print(f"Final dataframe shape: {df.shape}")

# Save column governance log
column_log = pd.DataFrame({
    'column': df.columns,
    'dtype': df.dtypes,
    'missing_rate': df.isnull().mean(),
    'nunique': df.nunique()
})
print("\nColumn summary:")
display(column_log)


Dropped 65 columns with missingness < 0.01
Dropped 11 columns with missingness > 0.99

Created 141 missing value flags
Final dataframe shape: (182462, 282)

Created 141 missing value flags
Final dataframe shape: (182462, 282)

Column summary:

Column summary:


,column,dtype,missing_rate,nunique
bureau_1_avg_days_credit,bureau_1_avg_days_credit,float64,0.059804,112039
bureau_1_avg_loan_types,bureau_1_avg_loan_types,float64,0.041252,1780
bureau_1_perc_active_lines,bureau_1_perc_active_lines,float64,0.041252,4531
bureau_1_perc_lines_in_use,bureau_1_perc_lines_in_use,float64,0.041252,4448
bureau_1_debt_ratio_max_limit,bureau_1_debt_ratio_max_limit,float64,0.299953,121775
...,...,...,...,...
days_since_most_recent_app_event_prior_to_first_due_date_missing,days_since_most_recent_app_event_prior_to_firs...,int64,0.000000,2
total_mobile_app_events_between_first_loan_and_first_due_date_missing,total_mobile_app_events_between_first_loan_and...,int64,0.000000,2
phone_carrier_missing,phone_carrier_missing,int64,0.000000,2
vendor_1_telegram_privacy_status_missing,vendor_1_telegram_privacy_status_missing,int64,0.000000,2


# 3. Risk Banding

Create risk bands from the underwriting score and verify FPD monotonicity across bands.

In [20]:
# Create risk bands using qcut
try:
    # Use uw_bucket for risk bands, mapping highest bucket to 1 (lowest risk), lowest to 6 (highest risk)
    unique_buckets = sorted(df['uw_bucket'].dropna().unique(), reverse=True)
    bucket_map = {bucket: i+1 for i, bucket in enumerate(unique_buckets)}
    df['risk_band'] = df['uw_bucket'].map(bucket_map)
except ValueError as e:
    print("Handling ties in UW score...")
    df['risk_band'] = pd.qcut(df[CONFIG["UW_SCORE_COL"]], 
                             q=6, 
                             labels=[1, 2, 3, 4, 5, 6],
                             duplicates='drop')
    
# Calculate FPD rate by risk band
risk_band_stats = df.groupby('risk_band').agg({
    CONFIG["TARGET_COL"]: ['count', 'mean'],
    CONFIG["UW_SCORE_COL"]: ['min', 'max']
}).round(4)

print("Risk band statistics:")
display(risk_band_stats)

# Verify monotonicity
fpd_rates = risk_band_stats[(CONFIG["TARGET_COL"], 'mean')]
is_monotonic = fpd_rates.is_monotonic_increasing

print(f"\nFPD rates are {'monotonic' if is_monotonic else 'NOT monotonic'} across risk bands")

# Plot FPD rate by risk band
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='risk_band', y=CONFIG["TARGET_COL"])
plt.title('FPD Rate by Risk Band')
plt.xlabel('Risk Band (1=Lowest Risk)')
plt.ylabel('FPD Rate')
plt.savefig('plots/fpd_by_risk_band_baseline.png', bbox_inches='tight')
plt.show()

KeyError: 'uw_bucket'

# 4. Categorical Feature Handling

Process categorical features using a tiered approach based on cardinality:
- Low (≤5 levels): One-hot encoding
- Mid (6-30 levels): Rare level grouping + one-hot + frequency
- High (>30 levels): Frequency + cross-fitted target encoding

In [ ]:
# Helper functions for categorical encoding
def get_categorical_cols(df: pd.DataFrame) -> List[str]:
    """Identify categorical columns excluding target and risk band."""
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    return [col for col in cat_cols 
            if col not in [CONFIG["TARGET_COL"], 'risk_band']]

def split_by_cardinality(df: pd.DataFrame, cat_cols: List[str]) -> Dict[str, List[str]]:
    """Split categorical columns by cardinality into low, mid, and high."""
    cardinality = {col: df[col].nunique() for col in cat_cols}
    
    return {
        'low': [col for col, k in cardinality.items() if k <= 5],
        'mid': [col for col, k in cardinality.items() if 5 < k <= 30],
        'high': [col for col, k in cardinality.items() if k > 30]
    }

def handle_rare_levels(df: pd.DataFrame, col: str, min_freq: float = 0.005) -> pd.Series:
    """Collapse rare categories into 'Other'."""
    value_counts = df[col].value_counts(normalize=True)
    rare_levels = value_counts[value_counts < min_freq].index
    return df[col].replace(dict.fromkeys(rare_levels, 'Other'))

def cross_fitted_target_encoding(df: pd.DataFrame, col: str, target: str, 
                               n_splits: int = 5, alpha: float = 100.0) -> pd.Series:
    """Perform cross-fitted target encoding with smoothing."""
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    encoded = pd.Series(index=df.index, dtype='float64')
    global_mean = df[target].mean()
    
    for train_idx, val_idx in kf.split(df):
        # Compute level means on training fold
        train_stats = df.iloc[train_idx].groupby(col)[target].agg(['count', 'sum'])
        smoothed_means = (train_stats['sum'] + alpha * global_mean) / (train_stats['count'] + alpha)
        
        # Map to validation fold
        encoded.iloc[val_idx] = df.iloc[val_idx][col].map(smoothed_means).fillna(global_mean)
    
    return encoded

# Process categorical columns
print("Processing categorical columns...")

# Get categorical columns and split by cardinality
cat_cols = get_categorical_cols(df)
cardinality_groups = split_by_cardinality(df, cat_cols)

# Initialize feature blocks
X_num = df.select_dtypes(include=['float64', 'int64']).copy()
X_cat_encoded = pd.DataFrame(index=df.index)

# 1. Handle low cardinality: one-hot encoding
if cardinality_groups['low']:
    print(f"\nOne-hot encoding {len(cardinality_groups['low'])} low-cardinality features")
    X_cat_encoded = pd.concat([
        X_cat_encoded,
        pd.get_dummies(df[cardinality_groups['low']], drop_first=True)
    ], axis=1)

# 2. Handle mid cardinality: collapse rare + one-hot + frequency
for col in cardinality_groups['mid']:
    print(f"\nProcessing mid-cardinality feature: {col}")
    # Collapse rare levels
    df[f"{col}_grouped"] = handle_rare_levels(df, col)
    
    # One-hot the grouped version
    dummies = pd.get_dummies(df[f"{col}_grouped"], prefix=col, drop_first=True)
    
    # Frequency encoding
    freq_enc = df[col].map(df[col].value_counts(normalize=True))
    freq_enc.name = f"{col}_freq"
    
    X_cat_encoded = pd.concat([X_cat_encoded, dummies, freq_enc], axis=1)

# 3. Handle high cardinality: frequency + target encoding
for col in cardinality_groups['high']:
    print(f"\nProcessing high-cardinality feature: {col}")
    
    # Frequency encoding
    freq_enc = df[col].map(df[col].value_counts(normalize=True))
    freq_enc.name = f"{col}_freq"
    
    # Target encoding
    target_enc = cross_fitted_target_encoding(
        df, col, CONFIG["TARGET_COL"], 
        n_splits=CONFIG["N_SPLITS"]
    )
    target_enc.name = f"{col}_te"
    
    X_cat_encoded = pd.concat([X_cat_encoded, freq_enc, target_enc], axis=1)

# Combine numeric and categorical features
X = pd.concat([X_num, X_cat_encoded], axis=1)
y = df[CONFIG["TARGET_COL"]].astype(int)

print(f"\nFinal feature matrix shape: {X.shape}")
print(f"Number of categorical features encoded: {len(X_cat_encoded.columns)}")

# 5. Exploratory Variable Scoring

Analyze feature relationships with target:
- Correlation analysis for numeric features
- Mutual information scoring
- Risk band-specific correlations

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# 1. Correlation Analysis
numeric_correlations = X.corr()[CONFIG["TARGET_COL"]].sort_values(ascending=False)
print("Top correlations with target:")
display(numeric_correlations.head(10))

# 2. Mutual Information
mi_scores = mutual_info_classif(X, y, random_state=RANDOM_SEED)
mi_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
print("\nTop mutual information scores:")
display(mi_series.head(10))

# 3. Risk Band-Specific Correlations
band_corrs = pd.DataFrame()
for band in sorted(df['risk_band'].unique()):
    mask = df['risk_band'] == band
    band_corrs[f'Band_{band}'] = X[mask].corrwith(y[mask])

# Plot heatmap of correlations by risk band
plt.figure(figsize=(12, 8))
sns.heatmap(band_corrs.loc[band_corrs.abs().max(axis=1).sort_values(ascending=False).head(15).index],
            center=0, cmap='RdBu_r', annot=True, fmt='.2f')
plt.title('Feature Correlations by Risk Band')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/correlation_heatmap_by_band.png', bbox_inches='tight')
plt.show()

# Save top features summary
feature_importance = pd.DataFrame({
    'correlation': numeric_correlations.abs(),
    'mutual_info': mi_series
}).sort_values('mutual_info', ascending=False)

print("\nTop features by both metrics:")
display(feature_importance.head(10))

# 6. Decision Tree Modeling

Train a compact decision tree to identify high-risk segments:

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=CONFIG["TRAIN_SIZE"],
    stratify=y,
    random_state=RANDOM_SEED
)

# Train decision tree
min_samples = int(len(X) * CONFIG["MIN_SAMPLES_LEAF"])
tree = DecisionTreeClassifier(
    max_depth=CONFIG["MAX_DEPTH"],
    min_samples_leaf=min_samples,
    class_weight='balanced',
    random_state=RANDOM_SEED
)
tree.fit(X_train, y_train)

# Evaluate performance
y_pred = tree.predict(X_test)
y_prob = tree.predict_proba(X_test)[:, 1]

print("Model Performance:")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print tree structure
print("\nDecision Tree Structure:")
print(export_text(tree, feature_names=list(X.columns)))

# Extract leaf information
def get_leaf_info(tree, X, y, feature_names):
    """Extract information about terminal leaves including paths and statistics."""
    n_nodes = tree.tree_.node_count
    children_left = tree.tree_.children_left
    children_right = tree.tree_.children_right
    feature = tree.tree_.feature
    threshold = tree.tree_.threshold
    
    # Initialize for path extraction
    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # (node_id, depth)
    
    # Compute node depths
    while len(stack) > 0:
        node_id, depth = stack.pop()
        node_depth[node_id] = depth
        
        is_split_node = children_left[node_id] != children_right[node_id]
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True
    
    leaf_info = []
    for node_id in range(n_nodes):
        if not is_leaves[node_id]:
            continue
            
        # Build the decision path
        path = []
        current_node = node_id
        parent_nodes = []
        
        while current_node != 0:
            # Find parent
            parent = -1
            for i in range(n_nodes):
                if children_left[i] == current_node or children_right[i] == current_node:
                    parent = i
                    break
            if parent == -1:
                break
                
            # Add to path
            is_left = children_left[parent] == current_node
            feature_name = feature_names[feature[parent]]
            threshold_value = threshold[parent]
            
            path.append(
                f"{feature_name} {'<=' if is_left else '>'} {threshold_value:.3f}"
            )
            current_node = parent
            parent_nodes.append(parent)
        
        # Get samples that end up in this leaf
        mask = np.ones(len(X), dtype=bool)
        for p in reversed(parent_nodes):
            if children_left[p] in parent_nodes + [node_id]:
                mask &= X[feature_names[feature[p]]] <= threshold[p]
            else:
                mask &= X[feature_names[feature[p]]] > threshold[p]
        
        # Compute leaf statistics
        leaf_samples = X[mask]
        leaf_targets = y[mask]
        
        leaf_info.append({
            'node_id': node_id,
            'depth': node_depth[node_id],
            'path': ' & '.join(reversed(path)),
            'n_samples': len(leaf_samples),
            'fpd_rate': leaf_targets.mean(),
            'purity': leaf_targets.mean(),
            'mask': mask
        })
    
    return pd.DataFrame(leaf_info)

# Get leaf information
leaf_df = get_leaf_info(tree, X, y, list(X.columns))

# Filter candidate leaves for removal
baseline_fpd = y.mean()
min_leaf_size = len(X) * CONFIG["MIN_LEAF_SIZE"]
candidate_leaves = leaf_df[
    (leaf_df['fpd_rate'] > baseline_fpd + CONFIG["MIN_FPD_LIFT"]) & 
    (leaf_df['n_samples'] >= min_leaf_size)
].copy()

# Sort by efficiency
candidate_leaves['efficiency'] = (candidate_leaves['fpd_rate'] - baseline_fpd) / (candidate_leaves['n_samples'] / len(X))
candidate_leaves = candidate_leaves.sort_values('efficiency', ascending=False)

print("\nCandidate leaves for filtering:")
display(candidate_leaves)

# Save leaf information
candidate_leaves.to_csv('outputs/leaf_summary.csv', index=False)

# Visualize leaf FPD rates
plt.figure(figsize=(12, 6))
leaf_sizes = leaf_df['n_samples'] / len(X)
plt.scatter(leaf_sizes, leaf_df['fpd_rate'], alpha=0.6)
plt.axhline(y=baseline_fpd, color='r', linestyle='--', label='Baseline FPD')
plt.axhline(y=baseline_fpd + CONFIG["MIN_FPD_LIFT"], 
            color='g', linestyle='--', 
            label=f'Baseline + {CONFIG["MIN_FPD_LIFT"]:.1%}')

for _, leaf in candidate_leaves.iterrows():
    plt.annotate(f"Leaf {leaf['node_id']}", 
                (leaf['n_samples']/len(X), leaf['fpd_rate']))

plt.xlabel('Leaf Size (% of total)')
plt.ylabel('FPD Rate')
plt.title('Leaf FPD Rates vs Size')
plt.legend()
plt.tight_layout()
plt.savefig('plots/leaf_fpd_rates.png', bbox_inches='tight')
plt.show()

# 7. Rule Evaluation Engine

Create utilities for evaluating rules and their cumulative impact:

In [ ]:
import re

def mask_from_rule(rule: str, df: pd.DataFrame) -> pd.Series:
    """Convert a boolean rule string into a mask on the dataframe."""
    # Split into individual conditions
    conditions = [c.strip() for c in rule.split('&')]
    
    # Parse each condition
    masks = []
    for condition in conditions:
        # Extract components with regex
        match = re.match(r'(\w+)\s*([<>=]+)\s*([\d.]+)', condition)
        if not match:
            raise ValueError(f"Invalid condition format: {condition}")
            
        var, op, val = match.groups()
        val = float(val)
        
        if var not in df.columns:
            raise ValueError(f"Column {var} not found in dataframe")
            
        # Create mask based on operator
        if op == '<=':
            mask = df[var] <= val
        elif op == '<':
            mask = df[var] < val
        elif op == '>=':
            mask = df[var] >= val
        elif op == '>':
            mask = df[var] > val
        elif op == '==':
            mask = df[var] == val
        else:
            raise ValueError(f"Unsupported operator: {op}")
            
        masks.append(mask)
    
    # Combine all conditions (AND)
    return pd.concat(masks, axis=1).all(axis=1)

def evaluate_rules(df: pd.DataFrame, y: pd.Series, rules: List[str]) -> pd.DataFrame:
    """Evaluate each rule independently."""
    results = []
    baseline_fpd = y.mean()
    
    for rule in rules:
        # Get samples to remove
        removed_mask = mask_from_rule(rule, df)
        kept_mask = ~removed_mask
        
        # Calculate metrics
        removed_count = removed_mask.sum()
        removed_pct = removed_count / len(df)
        new_fpd = y[kept_mask].mean()
        fpd_reduction_pp = (baseline_fpd - new_fpd)
        removed_purity = y[removed_mask].mean()
        efficiency = fpd_reduction_pp / removed_pct if removed_pct > 0 else 0
        
        results.append({
            'rule': rule,
            'removed_count': removed_count,
            'removed_pct': removed_pct,
            'new_fpd_rate': new_fpd,
            'fpd_reduction_pp': fpd_reduction_pp,
            'removed_purity': removed_purity,
            'efficiency': efficiency
        })
    
    return pd.DataFrame(results)

def evaluate_rules_cumulative(df: pd.DataFrame, y: pd.Series, 
                            rules_sorted: List[str], 
                            risk_bands: pd.Series) -> pd.DataFrame:
    """Evaluate rules applied progressively."""
    results = []
    all_bands = sorted(risk_bands.unique())
    
    # Baseline metrics
    baseline = {
        'stage': 'Baseline',
        'rules_applied': 0,
        'kept_n': len(df),
        'kept_pct': 1.0,
        'overall_fpd': y.mean(),
        'bad_n': y.sum(),
        'good_n': len(y) - y.sum()
    }
    
    # Add band-specific FPD rates
    for band in all_bands:
        mask = risk_bands == band
        baseline[f'band_{band}_fpd'] = y[mask].mean()
    
    results.append(baseline)
    
    # Progressive application
    kept_mask = pd.Series(True, index=df.index)
    
    for i, rule in enumerate(rules_sorted, 1):
        # Apply new rule
        remove_mask = mask_from_rule(rule, df)
        kept_mask &= ~remove_mask
        
        # Calculate metrics
        kept_y = y[kept_mask]
        new_fpd = kept_y.mean()
        
        stage = {
            'stage': f'After R{i}',
            'rules_applied': i,
            'kept_n': kept_mask.sum(),
            'kept_pct': kept_mask.mean(),
            'overall_fpd': new_fpd,
            'bad_n': kept_y.sum(),
            'good_n': len(kept_y) - kept_y.sum()
        }
        
        # Band-specific FPD
        for band in all_bands:
            band_mask = risk_bands == band
            stage[f'band_{band}_fpd'] = y[kept_mask & band_mask].mean()
        
        results.append(stage)
        
        # Check stopping condition
        if i > 1:
            fpd_reduction = results[-2]['overall_fpd'] - new_fpd
            if fpd_reduction < CONFIG["MIN_FPD_REDUCTION"]:
                print(f"Stopping at {i} rules: FPD reduction below threshold")
                break
    
    return pd.DataFrame(results)

# Get rules from candidate leaves
rules = candidate_leaves['path'].tolist()

# Evaluate rules independently
single_rule_impact = evaluate_rules(df, y, rules)
print("Single rule impact:")
display(single_rule_impact)

# Evaluate rules cumulatively
cumulative_impact = evaluate_rules_cumulative(df, y, rules, df['risk_band'])
print("\nCumulative impact:")
display(cumulative_impact)

# Save results
single_rule_impact.to_csv('outputs/single_rule_impact.csv', index=False)
cumulative_impact.to_csv('outputs/cumulative_stages.csv', index=False)

# Create band FPD matrix
band_cols = [col for col in cumulative_impact.columns if col.startswith('band_')]
band_fpd_matrix = cumulative_impact[['stage'] + band_cols]
band_fpd_matrix.to_csv('outputs/band_fpd_matrix.csv', index=False)

# 8. Visualizations

Create comprehensive visualization suite for rule impact analysis:

In [ ]:
# Set up the visualization style
plt.style.use('seaborn')
colors = plt.cm.Set2(np.linspace(0, 1, 8))

# 1. Trade-off curve
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# FPD Rate
ax1.bar(cumulative_impact['stage'], 
        cumulative_impact['overall_fpd'] * 100,
        color=colors[0])
ax1.set_title('Overall FPD Rate by Stage')
ax1.set_ylabel('FPD Rate (%)')
for i, v in enumerate(cumulative_impact['overall_fpd']):
    ax1.text(i, v * 100, f'{v:.1%}', ha='center', va='bottom')

# Kept Percentage
ax2.bar(cumulative_impact['stage'],
        cumulative_impact['kept_pct'] * 100,
        color=colors[1])
ax2.set_title('Population Kept by Stage')
ax2.set_ylabel('Kept (%)')
for i, v in enumerate(cumulative_impact['kept_pct']):
    ax2.text(i, v * 100, f'{v:.1%}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('plots/tradeoff_curve.png', bbox_inches='tight')
plt.show()

# 2. Single Rule Impact
plt.figure(figsize=(12, 6))
rules_short = [f"R{i+1}" for i in range(len(single_rule_impact))]
plt.bar(rules_short, 
        single_rule_impact['fpd_reduction_pp'] * 100,
        color=colors[2])
plt.title('FPD Reduction by Individual Rule')
plt.ylabel('FPD Reduction (percentage points)')
plt.xlabel('Rule')

# Annotate with removed percentage
for i, (red, rem) in enumerate(zip(single_rule_impact['fpd_reduction_pp'],
                                 single_rule_impact['removed_pct'])):
    plt.text(i, red * 100, f'{rem:.1%}\nremoved', 
             ha='center', va='bottom')

plt.tight_layout()
plt.savefig('plots/single_rule_impact.png', bbox_inches='tight')
plt.show()

# 3. Stacked Bars of Marginal Removals
removal_data = []
baseline = {'good': len(y) - y.sum(), 'bad': y.sum()}

for i in range(len(cumulative_impact) - 1):
    curr = cumulative_impact.iloc[i]
    next_stage = cumulative_impact.iloc[i + 1]
    
    removed_good = curr['good_n'] - next_stage['good_n']
    removed_bad = curr['bad_n'] - next_stage['bad_n']
    
    removal_data.append({
        'stage': f"R{i+1}",
        'Goods Removed': removed_good,
        'Bads Removed': removed_bad
    })

removal_df = pd.DataFrame(removal_data)

plt.figure(figsize=(12, 6))
bottom = np.zeros(len(removal_df))

for col, color in zip(['Goods Removed', 'Bads Removed'], colors[3:5]):
    plt.bar(removal_df['stage'], removal_df[col], 
            bottom=bottom, label=col, color=color)
    bottom += removal_df[col]

plt.title('Marginal Removals by Rule')
plt.ylabel('Number of Applications')
plt.legend()
plt.tight_layout()
plt.savefig('plots/marginal_removals.png', bbox_inches='tight')
plt.show()

# 4. FPD by Risk Band
band_cols = [col for col in cumulative_impact.columns if col.startswith('band_')]
band_data = cumulative_impact[['stage'] + band_cols].melt(
    id_vars=['stage'],
    var_name='band',
    value_name='fpd_rate'
)
band_data['band'] = band_data['band'].str.extract('band_(\d+)').astype(int)

plt.figure(figsize=(12, 6))
for stage in band_data['stage'].unique():
    stage_data = band_data[band_data['stage'] == stage]
    plt.plot(stage_data['band'], stage_data['fpd_rate'] * 100,
             marker='o', label=stage)

plt.title('FPD Rate by Risk Band and Stage')
plt.xlabel('Risk Band')
plt.ylabel('FPD Rate (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.savefig('plots/fpd_by_band.png', bbox_inches='tight')
plt.show()

# 5. Volume Change Waterfall
baseline_volume = len(y)
volume_changes = []
last_volume = baseline_volume

for i in range(1, len(cumulative_impact)):
    curr_volume = cumulative_impact.iloc[i]['kept_n']
    change = curr_volume - last_volume
    volume_changes.append({
        'stage': f"R{i}",
        'change': change,
        'pct_change': change / baseline_volume
    })
    last_volume = curr_volume

change_df = pd.DataFrame(volume_changes)

plt.figure(figsize=(12, 6))
plt.bar(change_df['stage'], 
        change_df['pct_change'] * 100,
        color=colors[7])
plt.title('Incremental Volume Change by Rule')
plt.ylabel('Change in Volume (%)')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.2)

for i, v in enumerate(change_df['pct_change']):
    plt.text(i, v * 100, f'{v:.1%}', ha='center', va='bottom' if v > 0 else 'top')

plt.tight_layout()
plt.savefig('plots/volume_waterfall.png', bbox_inches='tight')
plt.show()

# 9. QA Checks and Final Summary

Perform final sanity checks and save summary metrics:

In [ ]:
# QA Checks
print("=== QA Checks ===")

# 1. Verify no post-application features used
if CONFIG["DROP_POST_APP_COLS"]:
    remaining_post_app = set(CONFIG["DROP_POST_APP_COLS"]) & set(df.columns)
    print(f"\nPost-application columns remaining: {remaining_post_app}")

# 2. Check encoding completeness
print(f"\nMissing values in feature matrix: {X.isnull().sum().sum()}")

# 3. Verify risk band consistency
print("\nRisk band distribution consistency:")
print(pd.DataFrame({
    'Initial': df['risk_band'].value_counts(normalize=True),
    'Final': df.loc[cumulative_impact.iloc[-1]['kept_mask'], 'risk_band'].value_counts(normalize=True)
}))

# Create final summary metrics
final_metrics = {
    'baseline': {
        'total_volume': len(df),
        'fpd_rate': y.mean(),
        'fpd_by_band': {
            f"band_{band}": y[df['risk_band'] == band].mean()
            for band in sorted(df['risk_band'].unique())
        }
    },
    'final': {
        'rules_applied': len(rules),
        'kept_volume': cumulative_impact.iloc[-1]['kept_n'],
        'kept_pct': cumulative_impact.iloc[-1]['kept_pct'],
        'final_fpd': cumulative_impact.iloc[-1]['overall_fpd'],
        'fpd_reduction_pp': (y.mean() - cumulative_impact.iloc[-1]['overall_fpd']),
        'fpd_by_band': {
            f"band_{band}": band_fpd_matrix.iloc[-1][f'band_{band}_fpd']
            for band in sorted(df['risk_band'].unique())
        }
    },
    'rules': {
        f"rule_{i+1}": {
            'definition': rule,
            'removed_pct': stats['removed_pct'],
            'fpd_reduction_pp': stats['fpd_reduction_pp']
        }
        for i, (rule, stats) in enumerate(zip(rules, single_rule_impact.to_dict('records')))
    }
}

# Save metrics to JSON
with open('outputs/metrics.json', 'w') as f:
    json.dump(final_metrics, f, indent=2)

# Save final rules in markdown format
with open('outputs/clean_rules.md', 'w') as f:
    f.write("# FPD Filtering Rules\n\n")
    f.write("Apply these rules in sequence. Remove applications matching ANY rule:\n\n")
    for i, rule in enumerate(rules, 1):
        f.write(f"{i}. `{rule}`\n")

print("\n=== Final Metrics ===")
print(f"Baseline FPD: {final_metrics['baseline']['fpd_rate']:.1%}")
print(f"Final FPD: {final_metrics['final']['final_fpd']:.1%}")
print(f"FPD Reduction: {final_metrics['final']['fpd_reduction_pp']:.1%}")
print(f"Volume Kept: {final_metrics['final']['kept_pct']:.1%}")
print(f"\nRules saved to 'clean_rules.md'")
print("Metrics saved to 'metrics.json'")